In [84]:
import pandas as pd
import pandera as pa

In [85]:
def extract(path: str) -> pd.DataFrame:
    df = pd.read_csv(path, encoding="utf-8")
    return df

path = "../bronze/rastreamento.csv"
df = extract(path)

In [86]:
df

,id_pacote,origem,destino,status_rastreamento,data_atualizacao
0,1001,São Paulo,Rio de Janeiro,EM TRÂNSITO,2024-05-20T10:00:00Z
1,1002,Belo Horizonte,Salvador,ENTREGUE,2024-05-18T14:30:00Z
2,1003,Curitiba,Porto Alegre,EXTRAVIADO,2024-05-19T08:15:00Z
3,1004,São Paulo,Brasília,EM TRÂNSITO,2024-05-20T11:45:00Z
4,1005,Recife,Fortaleza,EM ROTA DE ENTREGA,2024-05-20T09:20:00Z
5,1006,Rio de Janeiro,São Paulo,ENTREGUE,2024-05-19T17:00:00Z
6,1007,Salvador,Belo Horizonte,EM TRÂNSITO,2024-05-20T13:00:00Z
7,1008,Fortaleza,Recife,EM TRÂNSITO,2024-05-20T10:30:00Z
8,1009,Brasília,Curitiba,AGUARDANDO RETIRADA,2024-05-20T09:00:00Z
9,1010,Porto Alegre,São Paulo,CANCELADO,2024-05-18T11:00:00Z


In [ ]:
# Verificando a dimensao do dataset
df.shape

(10, 5)

In [ ]:
# Verificando as colunas presentes no DataFrame e os seus respectivos tipos
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 10 entries, 0 to 9
Data columns (total 5 columns):
 #   Column               Non-Null Count  Dtype 
---  ------               --------------  ----- 
 0   id_pacote            10 non-null     int64 
 1   origem               10 non-null     object
 2   destino              10 non-null     object
 3   status_rastreamento  10 non-null     object
 4   data_atualizacao     10 non-null     object
dtypes: int64(1), object(4)
memory usage: 532.0+ bytes


In [ ]:
# Verificando valores Not a Number (NaN), Nulos e duplicados
df.isna().sum(), df.isnull().sum(), df.duplicated().sum()

(id_pacote              0
 origem                 0
 destino                0
 status_rastreamento    0
 data_atualizacao       0
 dtype: int64,
 id_pacote              0
 origem                 0
 destino                0
 status_rastreamento    0
 data_atualizacao       0
 dtype: int64,
 np.int64(0))

In [ ]:
# Entendendo os valores possiveis para o status do rastreamento
df['status_rastreamento'].value_counts()

status_rastreamento
EM TRÂNSITO            4
ENTREGUE               2
EXTRAVIADO             1
EM ROTA DE ENTREGA     1
AGUARDANDO RETIRADA    1
CANCELADO              1
Name: count, dtype: int64

In [ ]:
# Transformando os dados e fazendo a verificacao
from pandera import Column, DataFrameSchema, Check, DateTime

def transform_and_validate(df: pd.DataFrame) -> pd.DataFrame:
    df['id_pacote'] = df['id_pacote'].astype(int)
    df['origem'] = df['origem'].astype("string")
    df['destino'] = df['destino'].astype("string")
    df['status_rastreamento'] = df['status_rastreamento'].astype("string")
    df['data_atualizacao'] = pd.to_datetime(df['data_atualizacao'], errors="coerce").dt.tz_localize(None)
    
    status_rastreamento = [
        "EM TRÂNSITO", 
        "ENTREGUE", 
        "EXTRAVIADO", 
        "EM ROTA DE ENTREGA", 
        "AGUARDANDO RETIRADA",
        "CANCELADO"
    ]

    schema = DataFrameSchema(
        {
            "id_pacote": Column(int, nullable=False),
            "origem": Column(str, nullable=False),
            "destino": Column(str, nullable=False),
            "status_rastreamento": Column(str, Check.isin(status_rastreamento), nullable=False),
            "data_atualizacao": Column(DateTime, nullable=False)
        }
    )
    try: 
        df = schema.validate(df)
        return df 
    except Exception as e:
        print(f"Exception was launched: {e}")
        return None

df = transform_and_validate(df)
df.dtypes

id_pacote                       int64
origem                 string[python]
destino                string[python]
status_rastreamento    string[python]
data_atualizacao       datetime64[ns]
dtype: object

In [95]:
from pathlib import Path 
def load(df: pd.DataFrame, path: str):
    try:
        df.to_csv(path, index=False, encoding="utf-8")
        return Path(path)
    except Exception as e:
        print(f"Erro salvando o CSV: {e}")
        return None 
    
arq = load(df, "rastreamento_tratado.csv")